# 05 动态网站抓取

内容导航：

1. Ajax和动态HTML
2. 动态爬虫1 - 爬取影评信息
3. Selenium
4. 动态爬虫2 - 爬取去哪网
5. PhantomJS*

## 5.1 Ajax和动态HTML

### Ajax

Ajax 即 Asynchronous JavaScript and XML，异步JS和XML，是基于JS的异步加载技术、XML（JSON）、DOM等技术的组合。使用Ajax技术无需刷新整个页面，只需对页面的局部更新，后台取回数据，减少了客户端（浏览器）和服务器之间的数据传输量，访问速度和用户体验得到提升。

### DHTML

DHTML是Dynamic HTML的简称，即动态HTML，是相对传统静态HTML而言的一种页面生成技术，也就是使用JavaScript和DOM，实现对页面内容的动态更新。

**如何爬取动态数据？**， 一般有两种做法：

1. 直接从JS中采集加载的数据
2. 直接采集浏览器中已经加载好的数据


## 5.2 动态爬虫1 - 爬取影评信息

下面以[MTime电影网发现频道《蜘蛛侠：英雄远征》](http://movie.mtime.com/238037/)为例介绍从JS中采集加载数据的方法：
1.使用浏览器的“开发者工具”进行分析。通过分析，在Network->JS分类中，找到包含Ajax字样的字符串，找到请求参数为：

http://service.library.mtime.com/Movie.api?Ajax_CallBack=true&Ajax_CallBackType=Mtime.Library.Services&Ajax_CallBackMethod=GetMovieOverviewRating&Ajax_CrossDomain=1&Ajax_RequestUrl=http%3A%2F%2Fmovie.mtime.com%2F238037%2F&t=201988233208252&Ajax_CallBackArgument0=238037

对应的评分和票房信息（使用json.loads反序列化为字典）为：

In [1]:
import json

movie_info = json.loads('{"value":{"isRelease":true,"movieRating":{"MovieId":238037,"RatingFinal":7.8,"RDirectorFinal":7.4,"ROtherFinal":7.6,"RPictureFinal":8.4,"RShowFinal":0,"RStoryFinal":7.3,"RTotalFinal":0,"Usercount":2565,"AttitudeCount":5046,"UserId":0,"EnterTime":0,"JustTotal":0,"RatingCount":0,"TitleCn":"","TitleEn":"","Year":"","IP":0},"movieTitle":"蜘蛛侠：英雄远征","tweetId":0,"userLastComment":"","userLastCommentUrl":"","releaseType":1,"boxOffice":{"Rank":17,"TotalBoxOffice":"13.80","TotalBoxOfficeUnit":"亿","TodayBoxOffice":"7.5","TodayBoxOfficeUnit":"万","ShowDays":42,"EndDate":"2019-08-08 23:30","FirstDayBoxOffice":"2.17","FirstDayBoxOfficeUnit":"亿"}},"error":null}')
movie_info

{'value': {'isRelease': True,
  'movieRating': {'MovieId': 238037,
   'RatingFinal': 7.8,
   'RDirectorFinal': 7.4,
   'ROtherFinal': 7.6,
   'RPictureFinal': 8.4,
   'RShowFinal': 0,
   'RStoryFinal': 7.3,
   'RTotalFinal': 0,
   'Usercount': 2565,
   'AttitudeCount': 5046,
   'UserId': 0,
   'EnterTime': 0,
   'JustTotal': 0,
   'RatingCount': 0,
   'TitleCn': '',
   'TitleEn': '',
   'Year': '',
   'IP': 0},
  'movieTitle': '蜘蛛侠：英雄远征',
  'tweetId': 0,
  'userLastComment': '',
  'userLastCommentUrl': '',
  'releaseType': 1,
  'boxOffice': {'Rank': 17,
   'TotalBoxOffice': '13.80',
   'TotalBoxOfficeUnit': '亿',
   'TodayBoxOffice': '7.5',
   'TodayBoxOfficeUnit': '万',
   'ShowDays': 42,
   'EndDate': '2019-08-08 23:30',
   'FirstDayBoxOffice': '2.17',
   'FirstDayBoxOfficeUnit': '亿'}},
 'error': None}

In [2]:
from myspider import HtmlDownloader

downloader = HtmlDownloader()
url = 'http://service.library.mtime.com/Movie.api?Ajax_CallBack=true&Ajax_CallBackType=Mtime.Library.Services&Ajax_CallBackMethod=GetMovieOverviewRating&Ajax_CrossDomain=1&Ajax_RequestUrl=http%3A%2F%2Fmovie.mtime.com%2F238037%2F&t=201988233208252&Ajax_CallBackArgument0=238037'
response = downloader.download(url)

In [3]:
import re
pattern = re.compile(r'=(.*?);')
result = re.findall(pattern, response)[0]

value = json.loads(result)
value = value.get('value').get('boxOffice')
value

{'Rank': 28,
 'TotalBoxOffice': '13.80',
 'TotalBoxOfficeUnit': '亿',
 'TodayBoxOffice': '0.8',
 'TodayBoxOfficeUnit': '万',
 'ShowDays': 43,
 'EndDate': '2019-08-09 09:45',
 'FirstDayBoxOffice': '2.17',
 'FirstDayBoxOfficeUnit': '亿'}

## 5.3 Selenium

上述分析Ajax请求的方法需要手工分析每个Ajax请求，这是一项非常繁重的工作，没有一定的JS功底，难以做到。而且有时请求参数是加密的，这使得手工分析几乎不可能。此时就需要第二种方法，好处就是直接提取浏览器渲染好的结果，无需进行Ajax请求分析。

### Selenium介绍

如何在Python程序中驱动浏览器从渲染好的页面中爬取数据呢？那就是使用可以模拟人与浏览器自动进行交互的程序。其中Selenium就是这样一款工具，此工具可以控制各种主流浏览器（如Chrome、Safari、Firefox），并且支持各种语言和平台。

### Selenium的安装

```
pip install selenium==3.01
```

除了Python库，还要安装浏览器驱动，不同浏览器需安装不同的驱动，如Chrome浏览器的驱动为chromedriver

In [4]:
!pip install selenium

In [2]:
!pip show selenium

Name: selenium
Version: 3.141.0
Summary: Python bindings for Selenium
Home-page: https://github.com/SeleniumHQ/selenium/
Author: UNKNOWN
Author-email: UNKNOWN
License: Apache 2.0
Location: /anaconda3/lib/python3.7/site-packages
Requires: urllib3
Required-by: 


### 快速入门

代码示例：

In [65]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time

driver = webdriver.Chrome()
driver.get('http://www.baidu.com')
elem = driver.find_element_by_name('wd')
time.sleep(3)
elem.clear()
elem.send_keys(u'网络爬虫')
elem.send_keys(Keys.RETURN)

### 元素选取

元素选取方法如下表所示：

![Selenium元素选取方法](images/selenium_selector.jpeg)

更通用的方法：

* find_element
* find_elements

In [66]:
from selenium.webdriver.common.by import By

elem = driver.find_element(By.XPATH, '//*[@id="kw"]')
elem.clear()
elem.send_keys(u'selenium')
elem.send_keys(Keys.RETURN)

## 动态爬虫2 - 爬取去哪网

In [69]:
url = 'https://hotel.qunar.com/'
driver.get(url)
driver.maximize_window()
driver.implicitly_wait(10)

In [70]:
# 定位要操作的网页元素
elem_to_city = driver.find_element(By.NAME, 'toCity')
elem_search = driver.find_element(By.CLASS_NAME, 'search-button')

elem_to_city.clear()
elem_to_city.send_keys(u'太原')

elem_search.click()

In [72]:
js = 'window.scrollTo(0, document.body.scrollHeight);'
driver.execute_script(js)

In [73]:
next_page = driver.find_element(By.XPATH, '//*[@id="searchHotelPanel"]/div[6]/div[1]/div/ul/li[10]/a')
next_page.click()

In [74]:
len(driver.page_source)

462884

In [60]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC